In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from tensorflow import keras
from tensorflow.keras import layers


df = pd.read_csv("cattle_health_data.csv")
print(df.head())
print(df.columns)


df.drop_duplicates(inplace=True)

for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].median(), inplace=True)




milk_col = [c for c in df.columns if 'milk' in c.lower()][0]

y = df[milk_col]
X = df.drop([milk_col], axis=1)


categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64','float64']).columns


preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

X_processed = preprocessor.fit_transform(X)

if hasattr(X_processed, "toarray"):
    X_processed = X_processed.toarray()


split = int(len(X_processed) * 0.8)

X_train = X_processed[:split]
X_test  = X_processed[split:]

y_train = y[:split]
y_test  = y[split:]


X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test  = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)



def transformer_block(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.MultiHeadAttention(
        key_dim=head_size,
        num_heads=num_heads,
        dropout=dropout
    )(inputs, inputs)

    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dense(inputs.shape[-1])(ff)

    return layers.LayerNormalization(epsilon=1e-6)(ff + x)


inputs = keras.Input(shape=(X_train.shape[1], 1))

x = transformer_block(inputs, head_size=32, num_heads=2, ff_dim=64)
x = layers.GlobalAveragePooling1D()(x)

x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1)(x)

model = keras.Model(inputs, outputs)


model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

model.summary()


history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=32
)


pred = model.predict(X_test)

mae = mean_absolute_error(y_test, pred)
rmse = np.sqrt(mean_squared_error(y_test, pred))
r2 = r2_score(y_test, pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)


plt.plot(history.history['loss'], label="Train")
plt.plot(history.history['val_loss'], label="Val")
plt.legend()
plt.title("Loss Curve")
plt.show()



plt.scatter(y_test, pred, alpha=0.5)
plt.xlabel("True")
plt.ylabel("Predicted")
plt.title("Transformer Predictions")
plt.show()


def predict_milk(df_input):
    proc = preprocessor.transform(df_input)

    if hasattr(proc, "toarray"):
        proc = proc.toarray()

    proc = proc.reshape(proc.shape[0], proc.shape[1], 1)

    return model.predict(proc)


model.save("milk_transformer_model.h5")
